# 🚀 FastAPI Server Debugging & Startup Notebook

## **Fix those damn connection errors!** 

This notebook will help you:
- ✅ Start FastAPI servers properly
- 🔍 Debug connection issues
- 🧪 Test API endpoints
- 🔧 Fix common problems

**The Issue:** Your Streamlit app can't connect to FastAPI servers because they're not running.

**The Solution:** Let's start them properly and make sure everything works!

## 1. 🔧 Environment Setup and Dependencies

First, let's make sure all required packages are installed and working.

In [ ]:
import os
import sys
import subprocess
import requests
import time
import signal
import psutil
import socket
from threading import Thread
import json

print("🔍 Checking Python environment...")
print(f"Python version: {sys.version}")
print(f"Working directory: {os.getcwd()}")

# Check if we're in virtual environment
if hasattr(sys, 'real_prefix') or (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix):
    print("✅ Virtual environment is active")
else:
    print("⚠️  No virtual environment detected")

# Check critical dependencies
try:
    import fastapi
    import uvicorn
    import streamlit
    import requests
    print("✅ Core dependencies found")
    print(f"   - FastAPI: {fastapi.__version__}")
    print(f"   - Uvicorn: {uvicorn.__version__}")
    print(f"   - Streamlit: {streamlit.__version__}")
    print(f"   - Requests: {requests.__version__}")
except ImportError as e:
    print(f"❌ Missing dependency: {e}")
    print("Run: pip install fastapi uvicorn streamlit requests")

## 2. 🔍 Server Status Checking

Let's check if the FastAPI servers are already running on ports 8000 and 8001.

In [ ]:
def check_port(port):
    """Check if a port is open/in use"""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        result = sock.connect_ex(('localhost', port))
        return result == 0

def check_server_health(port, endpoint="/"):
    """Check if server is responding to HTTP requests"""
    try:
        response = requests.get(f"http://localhost:{port}{endpoint}", timeout=5)
        return response.status_code == 200
    except:
        return False

def get_processes_on_port(port):
    """Get processes running on a specific port"""
    processes = []
    for proc in psutil.process_iter(['pid', 'name', 'cmdline']):
        try:
            for conn in proc.connections():
                if conn.laddr.port == port:
                    processes.append({
                        'pid': proc.info['pid'],
                        'name': proc.info['name'],
                        'cmdline': ' '.join(proc.info['cmdline']) if proc.info['cmdline'] else ''
                    })
        except:
            continue
    return processes

# Check server status
print("🔍 Checking server status...")
for port, name in [(8000, "Main API Server"), (8001, "Chat API Server")]:
    port_open = check_port(port)
    server_healthy = check_server_health(port)
    processes = get_processes_on_port(port)
    
    print(f"\n📡 {name} (Port {port}):")
    print(f"   Port open: {'✅' if port_open else '❌'}")
    print(f"   HTTP responding: {'✅' if server_healthy else '❌'}")
    
    if processes:
        print(f"   Running processes:")
        for proc in processes:
            print(f"     - PID {proc['pid']}: {proc['name']} ({proc['cmdline'][:100]}...)")
    else:
        print(f"   No processes found on port {port}")

## 3. 🚀 Manual Server Startup

Let's start the FastAPI servers manually and monitor their startup process.

In [ ]:
import subprocess
import threading
import time

class ServerManager:
    def __init__(self):
        self.processes = {}
        
    def start_server(self, script_path, port, name):
        """Start a FastAPI server with detailed logging"""
        print(f"🚀 Starting {name} on port {port}...")
        
        # Change to the correct directory
        server_dir = os.path.join(os.getcwd(), "florence2")
        
        try:
            # Start the server process
            process = subprocess.Popen(
                [sys.executable, script_path],
                cwd=server_dir,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True,
                bufsize=1,
                universal_newlines=True
            )
            
            self.processes[name] = process
            
            # Monitor startup in a separate thread
            def monitor_startup():
                start_time = time.time()
                while time.time() - start_time < 30:  # 30 second timeout
                    if check_port(port):
                        print(f"✅ {name} started successfully! PID: {process.pid}")
                        return
                    time.sleep(1)
                print(f"⏰ {name} startup timeout")
            
            threading.Thread(target=monitor_startup, daemon=True).start()
            
            # Log first few lines of output
            def log_output():
                for i, line in enumerate(process.stdout):
                    if i < 10:  # Only show first 10 lines
                        print(f"   [{name}] {line.strip()}")
                    if "Uvicorn running" in line:
                        break
            
            threading.Thread(target=log_output, daemon=True).start()
            
            return process
            
        except Exception as e:
            print(f"❌ Failed to start {name}: {e}")
            return None
    
    def stop_server(self, name):
        """Stop a server process"""
        if name in self.processes:
            process = self.processes[name]
            process.terminate()
            try:
                process.wait(timeout=5)
                print(f"🛑 {name} stopped")
            except subprocess.TimeoutExpired:
                process.kill()
                print(f"🔪 {name} force killed")
            del self.processes[name]
    
    def stop_all(self):
        """Stop all running servers"""
        for name in list(self.processes.keys()):
            self.stop_server(name)

# Create server manager
server_manager = ServerManager()
print("Server manager ready!")

In [ ]:
# Start both servers
print("🔥 Starting FastAPI servers...")

# Start main API server (port 8000)
main_server = server_manager.start_server("main.py", 8000, "Main API Server")

# Wait a bit for the first server to start
time.sleep(3)

# Start chat API server (port 8001)  
chat_server = server_manager.start_server("chatapi.py", 8001, "Chat API Server")

print("\n⏳ Waiting for servers to fully start...")
time.sleep(5)

# Check if servers are running
print("\n📊 Final status check:")
for port, name in [(8000, "Main API Server"), (8001, "Chat API Server")]:
    if check_port(port):
        print(f"✅ {name} is running on port {port}")
        if check_server_health(port, "/docs"):
            print(f"   📖 API docs available at: http://localhost:{port}/docs")
    else:
        print(f"❌ {name} is NOT running on port {port}")

## 4. 🧪 API Endpoint Testing

Now let's test the API endpoints to make sure they're working correctly.

In [ ]:
def test_api_endpoint(url, method="GET", data=None, files=None):
    """Test an API endpoint"""
    try:
        if method == "GET":
            response = requests.get(url, timeout=10)
        elif method == "POST":
            response = requests.post(url, data=data, files=files, timeout=10)
        
        print(f"✅ {method} {url}")
        print(f"   Status: {response.status_code}")
        print(f"   Response: {response.text[:200]}...")
        return response
    except requests.exceptions.ConnectionError:
        print(f"❌ Connection failed to {url}")
        print("   Server might not be running!")
    except Exception as e:
        print(f"❌ Error testing {url}: {e}")
    return None

# Test basic endpoints
print("🧪 Testing API endpoints...")

# Test main server
print("\n📡 Testing Main API Server (Port 8000):")
test_api_endpoint("http://localhost:8000/")
test_api_endpoint("http://localhost:8000/docs")

# Test chat server
print("\n💬 Testing Chat API Server (Port 8001):")
test_api_endpoint("http://localhost:8001/")
test_api_endpoint("http://localhost:8001/docs")

In [ ]:
# Test the specific endpoints that were failing
print("🔥 Testing problematic endpoints...")

# Create a simple test file for upload
test_content = b"This is a test document for OCR processing."
test_file = ('test.txt', test_content, 'text/plain')

print("\n📄 Testing OCR endpoint (Port 8001):")
ocr_url = "http://localhost:8001/extract_text/"
ocr_data = {"ocr_method": "tesseract"}
files = {"file": test_file}

ocr_response = test_api_endpoint(ocr_url, "POST", data=ocr_data, files=files)

print("\n🎯 Testing Knowledge Object endpoint (Port 8000):")
ko_url = "http://localhost:8000/upload-file/"
ko_data = {
    "model_name": "google",
    "ocr_method": "tesseract"
}
files = {"file": test_file}

ko_response = test_api_endpoint(ko_url, "POST", data=ko_data, files=files)

# Test chat endpoint with JSON data
print("\n💬 Testing Chat endpoint (Port 8001):")
chat_url = "http://localhost:8001/search_and_respond/"
chat_data = {
    "session_uuid": "123e4567-e89b-12d3-a456-426614174000",
    "extracted_text": "Test document content",
    "prompt": "Hello, this is a test",
    "model_name": "google",
    "search_method": "Embedding + Qdrant",
    "conversation_history": []
}

try:
    chat_response = requests.post(chat_url, json=chat_data, timeout=10)
    print(f"✅ POST {chat_url}")
    print(f"   Status: {chat_response.status_code}")
    print(f"   Response: {chat_response.text[:200]}...")
except Exception as e:
    print(f"❌ Chat endpoint error: {e}")

## 5. 🔍 Connection Error Diagnosis

Let's identify and fix common connection issues.

In [ ]:
def diagnose_connection_issues():
    """Diagnose common connection problems"""
    print("🔍 Diagnosing connection issues...")
    
    # Check if files exist
    script_files = [
        "florence2/main.py",
        "florence2/chatapi.py"
    ]
    
    print("\n📄 Checking server files:")
    for file_path in script_files:
        if os.path.exists(file_path):
            print(f"✅ {file_path} exists")
        else:
            print(f"❌ {file_path} NOT FOUND")
            
    # Check for common port conflicts
    print("\n🔌 Checking for port conflicts:")
    common_ports = [8000, 8001, 3000, 5000, 8080]
    for port in common_ports:
        processes = get_processes_on_port(port)
        if processes:
            print(f"⚠️  Port {port} in use by:")
            for proc in processes:
                print(f"     - {proc['name']} (PID: {proc['pid']})")
        else:
            print(f"✅ Port {port} available")
    
    # Check localhost connectivity  
    print("\n🌐 Testing localhost connectivity:")
    try:
        response = requests.get("http://localhost", timeout=5)
        print("✅ Localhost is reachable")
    except:
        print("❌ Localhost connectivity issues")
    
    # Check if antivirus/firewall might be blocking
    print("\n🛡️  Common blocking issues to check:")
    print("   - Windows Firewall blocking Python")
    print("   - Antivirus software blocking server startup")
    print("   - Corporate proxy settings")
    print("   - VPN interfering with localhost")

# Run diagnostics
diagnose_connection_issues()

## 6. ⚙️ Alternative Server Configuration

If the default setup isn't working, let's try alternative configurations.

In [ ]:
# Try alternative ports if 8000/8001 are problematic
alternative_ports = {
    "main": 8002,
    "chat": 8003
}

def start_server_with_uvicorn(script_name, port, app_name="app"):
    """Start server using uvicorn directly"""
    print(f"🔄 Starting {script_name} on port {port} using uvicorn...")
    
    server_dir = os.path.join(os.getcwd(), "florence2")
    
    try:
        # Use uvicorn command directly
        cmd = [
            sys.executable, "-m", "uvicorn", 
            f"{script_name.replace('.py', '')}:{app_name}",
            "--host", "127.0.0.1",
            "--port", str(port),
            "--reload"
        ]
        
        process = subprocess.Popen(
            cmd,
            cwd=server_dir,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        
        # Wait for startup
        time.sleep(5)
        
        if check_port(port):
            print(f"✅ Server started on port {port}")
            return process
        else:
            print(f"❌ Server failed to start on port {port}")
            return None
            
    except Exception as e:
        print(f"❌ Error starting server: {e}")
        return None

print("🔄 Trying alternative server startup method...")

# Stop existing servers first
server_manager.stop_all()
time.sleep(2)

# Try starting with uvicorn directly
alt_main = start_server_with_uvicorn("main.py", alternative_ports["main"])
alt_chat = start_server_with_uvicorn("chatapi.py", alternative_ports["chat"])

if alt_main or alt_chat:
    print(f"\n📝 Update your Streamlit app URLs to:")
    if alt_main:
        print(f"   Main API: http://localhost:{alternative_ports['main']}")
    if alt_chat:
        print(f"   Chat API: http://localhost:{alternative_ports['chat']}")
else:
    print("❌ Alternative startup also failed")

## 7. 🎯 Streamlit Integration Testing

Let's test the connection between Streamlit and FastAPI to make sure they work together.

In [ ]:
# Simulate the exact requests that Streamlit makes
def test_streamlit_integration():
    """Test the exact API calls that Streamlit is trying to make"""
    print("🎯 Testing Streamlit integration...")
    
    # Test the problematic URLs from your error messages
    urls_to_test = [
        "http://localhost:8000/upload-file/",
        "http://localhost:8001/extract_text/",
        "http://localhost:8001/search_and_respond/"
    ]
    
    for url in urls_to_test:
        print(f"\n🔗 Testing {url}")
        try:
            # Just test if the endpoint is reachable (not sending actual data)
            response = requests.get(url.replace("upload-file/", "docs").replace("extract_text/", "docs").replace("search_and_respond/", "docs"), timeout=5)
            print(f"✅ Endpoint reachable (docs page): {response.status_code}")
        except requests.exceptions.ConnectionError:
            print(f"❌ Connection refused - server not running!")
        except Exception as e:
            print(f"⚠️  Other error: {e}")

test_streamlit_integration()

# Final status and instructions
print("\n" + "="*60)
print("🎉 SERVER STARTUP SUMMARY")
print("="*60)

working_ports = []
for port in [8000, 8001, alternative_ports["main"], alternative_ports["chat"]]:
    if check_port(port):
        working_ports.append(port)
        print(f"✅ Server running on port {port}")

if working_ports:
    print(f"\n🚀 SUCCESS! {len(working_ports)} server(s) are running!")
    print("\n📋 Next steps:")
    print("1. Keep this notebook running (don't close it)")
    print("2. Open a new terminal")
    print("3. Run: streamlit run florence2/mainpage.py")
    print("4. Your Streamlit app should now work!")
    
    if 8000 not in working_ports or 8001 not in working_ports:
        print("\n⚠️  NOTE: If using alternative ports, update these files:")
        print("   - florence2/pages/kocreation.py (change localhost:8000)")
        print("   - florence2/pages/Docchat.py (change localhost:8001)")
else:
    print("\n❌ NO SERVERS ARE RUNNING!")
    print("\n🔧 Troubleshooting steps:")
    print("1. Check if Python files exist in florence2/ directory")
    print("2. Make sure virtual environment is activated")
    print("3. Install missing dependencies")
    print("4. Check for firewall/antivirus blocking")
    print("5. Try running servers manually in terminal")

## 🚨 Emergency Server Controls

Run these cells if you need to restart or stop servers.

In [ ]:
# 🛑 STOP ALL SERVERS
print("🛑 Stopping all servers...")
server_manager.stop_all()

# Kill any remaining processes on ports 8000, 8001
for port in [8000, 8001, 8002, 8003]:
    processes = get_processes_on_port(port)
    for proc in processes:
        try:
            process = psutil.Process(proc['pid'])
            process.terminate()
            print(f"🔪 Killed process {proc['pid']} on port {port}")
        except:
            pass

time.sleep(2)
print("✅ All servers stopped")